# Problem 1

## a) We edit the underlying methods to become numerically stable.
The original code is numerically instable. To help with this, I introduced several new classes: `LoggedDiscrete`, which implements numerically stable random variable operations, and `LoggedFNode`/`LoggedVNode` classes for numerically stable message passing using the log-sum-exp methodology. 

The `get_beliefs` method, being just a couple of `for` loops, is more or less the same as the original, since a better way to utilize the underlying class methods was not obvious to me. 

In [28]:
import numpy as np 
import networkx as nx
from fglib import graphs, nodes, rv, inference

class LoggedDiscrete(rv.Discrete):
    """
    Same as discrete RV, but with pmf expressed as logarithms
    
    We override the unity, __mul__, normalize, marginalize, argmax methods
    Notably not overridden is the maximize method
    """
    
    def __init__(self, raw_pmf, *args):
        super().__init__(raw_pmf, *args)
    
    @classmethod
    def unity(cls, *args):
        """Initialize unit element of a discrete random variable.
        Args:
            *args: Instances of the class VNode representing the variables of
                the probability mass function. The number of the positional
                arguments must match the number of dimensions of the Numpy
                array.
        Raises:
            ParameterException: An error occurred initializing with invalid
                parameters.
        """
        n = len(args)
        return cls(np.zeros((1,) * n), *args)
    
    
    def __mul__(self, other):
        """Multiply other with self and return the result.
        Args:
            other: Multiplier for the discrete random variable.
        Returns:
            A new discrete random variable representing the multiplication.
        """
        # Verify dimensions of multiplicand and multiplier.
        if len(self.dim) < len(other.dim):
            self._expand(other.dim, other.pmf.shape)
        elif len(self.dim) > len(other.dim):
            other._expand(self.dim, self.pmf.shape)

        pmf = self.pmf + other.pmf

        return LoggedDiscrete(pmf, *self.dim)
    
#     def __add__(self, other):
#         return self.__mul__(other)
    
    
    def normalize(self):
        """Normalize probability mass function."""
        m = np.max(self.pmf)
        # subtract max to avoid numerical underflow
        pmf = np.exp(self.pmf - m)
        # normalize
        pmf = pmf / np.abs(np.sum(pmf))
        # back to logs
        pmf = np.log(pmf)
        return LoggedDiscrete(pmf, *self.dim)
    
    
    def marginalize(self, *dims, normalize=True):
        """Return the marginal for given dimensions.
        The probability mass function of the discrete random variable
        is marginalized along the given dimensions.
        Args:
            *dims: Instances of discrete random variables, which should be
                marginalized out.
            normalize: Boolean flag if probability mass function should be
                normalized after marginalization.
        Returns:
            A new discrete random variable representing the marginal.
        """
        axis = tuple(idx for idx, d in enumerate(self.dim) if d in dims)
#         print('Premarginalization is')
#         print(np.exp(self.pmf))
        m = np.max(self.pmf)
        e = np.exp(self.pmf - m)
        pmf = np.sum(e, axis)
        pmf = np.log(pmf) + m
#         print('Post marginalization is ')
#         print(np.exp(pmf))
        
#         pmf = np.sum(self.pmf, axis)
        if normalize:
            m = np.max(pmf)
            e = np.exp(pmf - m)
            pmf = np.log(e / np.sum(e)) + m

        new_dims = tuple(d for d in self.dim if d not in dims)
        return LoggedDiscrete(pmf, *new_dims)
    
    
    def argmax(self, dim=None):
        """Return the dimension index of the maximum.
        Args:
            dim: An optional discrete random variable along a marginalization
                should be performed and the maximum is searched over the
                remaining dimensions. In the case of None, the maximum is
                search along all dimensions.
        Returns:
            An integer representing the dimension of the maximum.
        """
        if dim is None:
            return np.unravel_index(self.pmf.argmax(), self.pmf.shape)
        m = self.marginalize(dim)
        return np.argmin(m.pmf)
    
    def from_discrete(cls, discrete):
        """
        Converts a discrete RV object to a LoggedDiscrete object
        """

In [29]:
class LoggedFNode(nodes.FNode):
    
    def __init__(self, label, factor=None):
        """Create a factor node."""
        super().__init__(label, factor)
    
    def spa(self, tnode):
        """Return message of the sum-product algorithm."""
        print('{} spa {}'.format(self, tnode))
        # Initialize with local factor
        msg = self.factor

        # Product over incoming messages
        for n in self.neighbors(tnode):
#             print('Starting message is') 
#             print(np.exp(msg.pmf))
#             print('With type')
#             print(type(msg))
            msgs = self.graph[n][self]['object'].get_message(n, self)
#             print('Added message is')
#             print(msgs)
#             print(np.exp(msgs.pmf))
#             print('With type')
#             print(type(msgs))
            msg *= self.graph[n][self]['object'].get_message(n, self)
#             print('Final message is ')
#             print(np.exp(msg.pmf))
#             print('Next loop now \n')
        
#         print('Type at marginalization')
#         print(type(msg))
        # Integration/Summation over incoming variables
        for n in self.neighbors(tnode):
            msg = msg.marginalize(n, normalize=False)

        return msg
    
class LoggedVNode(nodes.VNode):
    
    def __init__(self, label, rv_type, observed=False):
        """Create a variable node."""
        super().__init__(label, rv_type, observed)
    
    def belief(self, normalize=True):
        """Return belief of the variable node.
        Args:
            normalize: Boolean flag if belief should be normalized.
        """
        print('computing beliefs for {}'.format(self))
        iterator = self.graph.neighbors(self)

        # Pick first node
        n = next(iterator)

        # Product over all incoming messages
        belief = self.graph[n][self]['object'].get_message(n, self)
        for n in iterator:
            belief *= self.graph[n][self]['object'].get_message(n, self)

        if normalize:
            belief = belief.normalize()

        return belief
    
    
    def maximum(self, normalize=True):
        """Return the maximum probability of the variable node.
        Args:
            normalize: Boolean flag if belief should be normalized.
        """
        b = self.belief(normalize)
        return np.amin(b.pmf)
    
    
    def argmax(self):
        """Return the argument for maximum probability of the variable node."""
        # In case of multiple occurrences of the maximum values,
        # the indices corresponding to the first occurrence are returned.
        b = self.belief()
        return b.argmin(self)
    
    
    def spa(self, tnode):
        """Return message of the sum-product algorithm."""
        print('{} spa {}'.format(self, tnode))
        if self.observed:
            return self.init
        else:
            # Initial message
            msg = self.init
#             print('Measuring pre type of vnode spa')
#             print(type(msg))
            # Product over incoming messages
            for n in self.neighbors(tnode):
#                 print('Adding message of type')
#                 print(type(self.graph[n][self]['object'].get_message(n, self)))
#                 print('For {} and {}'.format(str(n), str(self)))
                msg *= self.graph[n][self]['object'].get_message(n, self)
#             print('Measuring post type of vnode spa')
#             print(type(msg))
#             print('\n')
            return msg
        

In [30]:
def get_beliefs(fg):
    """Belief propagation.
    Perform exact inference on tree structured fgs.
    Return the belief of all query_nodes.
    """ 
    # pick a random RV to start
    from random import choice
    query_node = choice(fg.get_vnodes())

    # Depth First Search to determine edges
    dfs = nx.dfs_edges(fg, query_node)

    # Convert tuple to reversed list
    backward_path = list(dfs)
    forward_path = reversed(backward_path)    

    # Messages in forward phase
    print('Starting forward phase')
    for (v, u) in forward_path:  # Edge direction: u -> v
        msg = u.spa(v)
        fg[u][v]['object'].set_message(u, v, msg)

    print('Starting backward phase')
    # Messages in backward phase
    for (u, v) in backward_path:  # Edge direction: u -> v
        msg = u.spa(v)
        fg[u][v]['object'].set_message(u, v, msg)

    # Retrieve marginal distributions
    beliefs = []
    for n in fg.get_vnodes():
        beliefs.append(n.belief())
    
    return beliefs

In [31]:
# Create factor graph
fg = graphs.FactorGraph()

# Create variable nodes
x1 = LoggedVNode("x1", LoggedDiscrete)
x2 = LoggedVNode("x2", LoggedDiscrete)
x3 = LoggedVNode("x3", LoggedDiscrete)
x4 = LoggedVNode("x4", LoggedDiscrete)

# Create factor nodes
f12 = LoggedFNode("f12")
f234 = LoggedFNode("f234")
f3 = LoggedFNode("f3")
f4 = LoggedFNode("f4")

# Add nodes to factor graph
fg.set_nodes([x1, x2, x3, x4])
fg.set_nodes([f12, f234, f3,f4 ])

# Add edges to factor graph
fg.set_edge(x1, f12)
fg.set_edge(f12, x2)
fg.set_edge(x2, f234)
fg.set_edge(f234, x3)
fg.set_edge(f234, x4)
fg.set_edge(x3, f3)
fg.set_edge(x4, f4)

#add potential for f_3: p(x3)
dist_f3 = np.log([0.5, 0.5]).tolist()
f3.factor = LoggedDiscrete(dist_f3,x3)

#add potential for f_4: p(x4)
dist_f4 = np.log([0.4,0.6]).tolist()
f4.factor = LoggedDiscrete(dist_f4,x4)

# add potential for f_{234}: p(x2, x3, x4) = p(x2|x3,x4) p(x3,x4)
px3x4=np.log(np.outer(np.exp(dist_f3),np.exp(dist_f4))) # WHAT is this
px3x4=np.reshape(px3x4, np.shape(px3x4)+(1,))
px2_conditioned_x3x4=np.log([[[0.2,0.8],
                     [0.25,0.75],],
                     [[0.7,0.3],
                     [0.3,0.7]]]).tolist()

dist_f234 =px3x4+px2_conditioned_x3x4 # __mul__
f234.factor = LoggedDiscrete(dist_f234,x3,x4,x2)

# add potential for f_{12}:  p (x1,x2) = p(x1 | x2) p(x2)
px1_conditioned_x2 = np.log([[0.5,0.5],
                     [0.7,0.3]]).tolist()
# px2= np.sum(dist_f234, axis=(0,1)) 
# REPLACED with marginalization
# use log-sum-exp
m = np.max(dist_f234)
px2 = np.log(np.exp(dist_f234 - m).sum(axis=(0,1))) + m
dist_f12 = px2[:,np.newaxis]+px1_conditioned_x2 # __mul__
f12.factor = LoggedDiscrete(dist_f12,x2,x1)
# Perform sum-product algorithm on factor graph
# and request belief of variable node x1
# belief = inference.sum_product(fg, x3)

## b) We print the marginal distributions of the 4 RVs below.

In [32]:
beliefs = get_beliefs(fg)
# Print belief of variable nodes
print("Belief of variable nodes ")
for belief in beliefs:
    print(np.exp(belief.pmf))

Starting forward phase
f3 spa x3
f4 spa x4
x4 spa f234
x1 spa f12
f12 spa x2
x2 spa f234
f234 spa x3
Starting backward phase
x3 spa f234
f234 spa x2
x2 spa f12
f12 spa x1
f234 spa x4
x4 spa f4
x3 spa f3
computing beliefs for x1
computing beliefs for x2
computing beliefs for x3
computing beliefs for x4
Belief of variable nodes 
[0.65897284 0.34102716]
[0.20513578 0.79486422]
[0.52640912 0.47359088]
[0.28679718 0.71320282]


### Note that the above, numerically stable code is the same as if we used the built-in `sum_product` method.
This is because the factor graph of the provided example is not large, so numerical stability is not an issue.

In [6]:
# Create factor graph
fg = graphs.FactorGraph()

# Create variable nodes
x1 = nodes.VNode("x1", rv.Discrete)
x2 = nodes.VNode("x2", rv.Discrete)
x3 = nodes.VNode("x3", rv.Discrete)
x4 = nodes.VNode("x4", rv.Discrete)

# Create factor nodes
f12 = nodes.FNode("f12", rv.Discrete)
f234 = nodes.FNode("f234", rv.Discrete)
f3 = nodes.FNode("f3", rv.Discrete)
f4 = nodes.FNode("f4", rv.Discrete)

# Add nodes to factor graph
fg.set_nodes([x1, x2, x3, x4])
fg.set_nodes([f12, f234, f3,f4 ])

# Add edges to factor graph
fg.set_edge(x1, f12)
fg.set_edge(f12, x2)
fg.set_edge(x2, f234)
fg.set_edge(f234, x3)
fg.set_edge(f234, x4)
fg.set_edge(x3, f3)
fg.set_edge(x4, f4)

#add potential for f_3: p(x3)
dist_f3 = [0.5, 0.5]
f3.factor = rv.Discrete(dist_f3,x3)

#add potential for f_4: p(x4)
dist_f4 = [0.4,0.6]
f4.factor = rv.Discrete(dist_f4,x4)

# add potential for f_{234}: p(x2, x3, x4) = p(x2|x3,x4) p(x3,x4)
px3x4=np.outer(dist_f3,dist_f4)
px3x4=np.reshape(px3x4, np.shape(px3x4)+(1,))
px2_conditioned_x3x4=[[[0.2,0.8],
                     [0.25,0.75],],
                     [[0.7,0.3],
                     [0.3,0.7]]]

dist_f234 =px3x4*px2_conditioned_x3x4
f234.factor = rv.Discrete(dist_f234,x3,x4,x2)

# add potential for f_{12}:  p (x1,x2) = p(x1 | x2) p(x2)
px1_conditioned_x2 = [[0.5,0.5],
                     [0.7,0.3]]
px2= np.sum(dist_f234, axis=(0,1))
dist_f12 =px2[:,np.newaxis]*px1_conditioned_x2
f12.factor = rv.Discrete(dist_f12,x2,x1)
# Perform sum-product algorithm on factor graph
# and request belief of variable node x1
belief = inference.sum_product(fg, x3)

In [7]:
for RV in [x1, x2, x3, x4]:
    print(inference.sum_product(fg, RV).pmf)

[0.65897284 0.34102716]
[0.20513578 0.79486422]
[0.52640912 0.47359088]
[0.28679718 0.71320282]


# Problem 2

## a) The code for constructing the factor graph from H is given below.

In [8]:
def make_factor_graph(H):
    """
    Construct a factor graph given a parity check matrix H.
    H is array-like
    Returns the factor graph
    """
    fg = graphs.FactorGraph()
    
    vnodes = []
    fnodes = []
    
    # Create variable nodes
    for i in range(H.shape[1]):
        vnodes.append(LoggedVNode('x' + str(i), LoggedDiscrete))
    
    # Create factor nodes
    for i in range(H.shape[0]):
        fnodes.append(LoggedFNode('f' + str(i), LoggedDiscrete))
    
    # Add nodes to factor graph
    fg.set_nodes(vnodes)
    fg.set_nodes(fnodes)
    
    # add edges to factor graph
    nonzeros = np.nonzero(H)
    for i in range(len(nonzeros[0])):
        fg.set_edge(fnodes[nonzeros[0][i]], vnodes[nonzeros[1][i]])
    
    # set potentials for factors
    # 1 if indices sum to 0
    # 0 otherwise
    for f in fnodes:
        n_neighbors = len(list(f.neighbors()))
        potential = np.zeros([2] * n_neighbors).astype('int8')
        for i in range(potential.size):
            # use the index's binary representation to determine odd or even
            # for the potential function, odds = 0, evens = 1
            binary = list('{0:b}'.format(i)) # get binary
            binary = [int(b) for b in binary] # cast to int
            potential.put(i, int(np.sum(binary) + 1) % 2) # determine potential based on odd or even
        f.factor = rv.Discrete(potential, *list(f.neighbors()))
    
    return fg

### We double check the first factor's PMF below.
As expected, with 4 nodes tied to the first factor, there are 4 dimensions in the corresponding factor PMF. Moreover, 1's are in the position where the indices sum to an even number, and 0's are in the position where the indices sum to an odd number. This is in accordance with the requirements of the LDPC code.

In [9]:
H = np.array([[1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1]])
fg_h = make_factor_graph(H)
print(fg_h.get_fnodes()[0].factor.pmf)
print('\n\n')
print(fg_h.get_fnodes()[1].factor.pmf)

[[[[1. 0.]
   [0. 1.]]

  [[0. 1.]
   [1. 0.]]]


 [[[0. 1.]
   [1. 0.]]

  [[1. 0.]
   [0. 1.]]]]



[[[[1. 0.]
   [0. 1.]]

  [[0. 1.]
   [1. 0.]]]


 [[[0. 1.]
   [1. 0.]]

  [[1. 0.]
   [0. 1.]]]]


## b) We construct the `H` and `G` matrices ourselves
by constructing them in canonical form. We demonstrate for `n=8` below.

In [10]:
import pyldpc

In [11]:
def hg(n, k):
    """
    Generate parity check matrix G and code matrix H in canonical form.
    
    n: half the message input length
    k: number of variables per factor
    """
    H = np.zeros([n, 2*n])
    for i in range(n):
        H[i, np.arange(i, i+k-1) % n] = 1
    H[:,n:] = np.identity(n)
    G = np.zeros([n, 2*n])
    G[:,:n] = np.identity(n)
    G[:,n:] = H[:,:n].T
    return H, G

H8, G8 = hg(8, 4)

In [12]:
H8

array([[1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [13]:
G8

array([[1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.]])

### We verify that these generate the 0 matrix

In [14]:
np.sum(np.abs(G8@H8.T % 2))

0.0

### We construct the 128-bit code.

In [15]:
H, G = hg(128, 4)

In [16]:
H.shape

(128, 256)

In [17]:
G.shape

(128, 256)

In [18]:
fg128 = make_factor_graph(H)

# create unary factor nodes 
# setting their pmf to [e, 1-e] for some e
e = 0.05
ufnodes = []
for v in fg128.get_vnodes():
    ufnodes.append(LoggedFNode('uf' + str(v), LoggedDiscrete))
    fg128.set_edge(ufnodes[-1], v)
    ufnodes[-1].factor = rv.Discrete([1 - e, e], v)
    
fg128.set_nodes(ufnodes)

In [19]:
list(fg128.get_fnodes())[128].factor.pmf

array([0.95, 0.05])

In [20]:
beliefs128 = get_beliefs(fg128)

Starting forward phase
ufx59 spa x59
ufx185 spa x185
x185 spa f57
ufx57 spa x57
ufx183 spa x183
x183 spa f55
ufx55 spa x55
ufx181 spa x181
x181 spa f53
ufx53 spa x53
ufx179 spa x179
x179 spa f51
ufx51 spa x51
ufx177 spa x177
x177 spa f49
ufx49 spa x49
ufx175 spa x175
x175 spa f47
ufx47 spa x47
ufx173 spa x173
x173 spa f45
ufx45 spa x45
ufx171 spa x171
x171 spa f43
ufx43 spa x43
ufx169 spa x169
x169 spa f41
ufx41 spa x41
ufx167 spa x167
x167 spa f39
ufx39 spa x39
ufx165 spa x165
x165 spa f37
ufx37 spa x37
ufx163 spa x163
x163 spa f35
ufx35 spa x35
ufx161 spa x161
x161 spa f33
ufx33 spa x33
ufx159 spa x159
x159 spa f31
ufx31 spa x31
ufx157 spa x157
x157 spa f29
ufx29 spa x29
ufx155 spa x155
x155 spa f27
ufx27 spa x27
ufx153 spa x153
x153 spa f25
ufx25 spa x25
ufx151 spa x151
x151 spa f23
ufx23 spa x23
ufx149 spa x149
x149 spa f21
ufx21 spa x21
ufx147 spa x147
x147 spa f19
ufx19 spa x19
ufx145 spa x145
x145 spa f17
ufx17 spa x17
ufx143 spa x143
x143 spa f15
ufx15 spa x15
ufx141 spa x141
x

AttributeError: 'NoneType' object has no attribute 'dim'

In [27]:
H

array([[1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1.]])

In [26]:
len(list(fg128.get_vnodes()))

8

# Depth first search is terminating upon seeing a loop

In [33]:
H, G = hg(4, 2)

fg128 = make_factor_graph(H)

# create unary factor nodes 
# setting their pmf to [e, 1-e] for some e
e = 0.05
ufnodes = []
for v in fg128.get_vnodes():
    ufnodes.append(LoggedFNode('uf' + str(v), LoggedDiscrete))
    fg128.set_edge(ufnodes[-1], v)
    ufnodes[-1].factor = rv.Discrete([1 - e, e], v)
    
fg128.set_nodes(ufnodes)

beliefs128 = get_beliefs(fg128)

Starting forward phase
ufx0 spa x0
ufx4 spa x4
x4 spa f0
f0 spa x0
Starting backward phase
x0 spa f0
f0 spa x4
x4 spa ufx4
x0 spa ufx0
computing beliefs for x0
computing beliefs for x1


TypeError: unsupported operand type(s) for *=: 'NoneType' and 'NoneType'